In [ ]:
from vast import *
launch('cuda_max_good >= 11.1 & gpu_name == "RTX 2080 Ti" & num_gpus == 4')
wait()
jittenate(local=True)

In [ ]:
import jittens
from experiments.architecture import load_opt_results

df = load_opt_results('fc-opt').xs('train', 1, 4).ewm(span=100).mean().min().rename('rv').reset_index()
check = df.set_index(['n', 'l', 'b', 'lr']).rv.notnull()
depths = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
widths = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
batches = [1024, 4*1024, 16*1024]
lrs = [1e-4, 8e-4, 64e-4, 512e-4]
for n in widths:
    for l in depths:
        for b in batches:
            for r in lrs:
                if (n, l, b, r) not in check.index:
                    jittens.submit(f'python -c "from experiments.architecture import *; run(\'fc-opt\', {n}, {l}, {b}, {r})" >logs.txt 2>&1', dir='.', resources={'gpu': 1})

In [ ]:
import time
import jittens
while not jittens.finished():
    jittens.manage()
    time.sleep(1)

In [ ]:
jittens.cancel()
jittens.cleanup()